- gigachain - основной инструмент построения пайплайнов с большими языковыми моделями;
- faiss-cpu - используем векторное хранилище FAISS для размещения чанков (фрагментов текстов для контекста)и эмбеддингов (векторных представлений) этих чанков;
- sentence-transformers - библиотека для локального использования эмбеддинговых моделей с ресурса Hugging Face;
- rank_bm25 - библиотека для работы с алгоритмами векторизации BM25 (вариации TF-IDF);
- playwright - библиотека для автоматизации браузера, в нашем случае для парсинга списка статей с веб-сайта.

In [ ]:
!playwright install

### Выполним сбор статей  
В качестве источника знаний используем статьи с научно-популярного сайта N + 1. Возьмем последние статьи по космонавтике.  

Используем AsyncChromiumLoader из GigaChain для загрузки страницы. Этот инструмент приходится использовать, потому что выборка статей по направлениям подгружает перечень статей не сразу, а значит получить его обычным запросом requests.get() не получится.  

Из-за асинхронной работы AsyncChromiumLoader сделаем сбор статей отдельным скриптом.  

В результате выполнения скрипта мы получим отдельный текстовый файл со ссылками на статьи по космической тематике.  

In [ ]:
%%writefile parser.py

from langchain_community.document_loaders import AsyncChromiumLoader
from bs4 import BeautifulSoup

# Инициализируем загрузчик
loader = AsyncChromiumLoader(["https://nplus1.ru/search?tags=871"])
html = loader.load()
articles = 'n1_climb_4 transition-colors duration-75 hover:text-main inline-block mb-10 sm:mb-5 font-spectral leading-24'
with open('page.html', 'w') as f:
  f.write(html[0].page_content)
soup = BeautifulSoup(html[0].page_content, 'html.parser')
links = []
with open('links.txt', 'w') as f:
  for link in soup.find_all('a', class_=articles):
    print(link)
    links.append(link['href'])
    f.write(link['href']+' ')

In [ ]:
!python parser.py

Откроем файл со ссылками.

In [ ]:
with open('links.txt') as f:
  sources = f.read()

print(sources.split())

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter